* Họ và tên: Nguyễn Tuấn Anh
* MSSV: 21520142
* Lớp: CS331.O11.KHTN
* Bài tập: Baitap02
* Ngày thực hiện: 05-10-2023

In [ ]:
import tensorflow as tf
#import all necessary layers
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
from tensorflow.keras import Model
from torchvision import datasets

In [ ]:
from tqdm import tqdm
import cv2
import numpy as np
from imutils import paths
import os

In [ ]:
#download dataset
path='./'
dataset =  datasets.Caltech101(path, download=True)

131740031it [00:00, 180943881.16it/s]


Extracting ./caltech101/101_ObjectCategories.tar.gz to ./caltech101


14028800it [00:00, 170681908.90it/s]


Extracting ./caltech101/Annotations.tar to ./caltech101


In [ ]:
#Extract Feature Histogram
image_paths = list(paths.list_images('./'))

data = []
labels = []
for img_path in tqdm(image_paths):
    label = img_path.split(os.path.sep)[-2]
    if label == "BACKGROUND_Google":
        continue
    img = cv2.imread(img_path)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, dsize=(224, 224))
    #hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    #size = img.shape[0]*img.shape[1]
    #hist = hist/size

    data.append(img)
    labels.append(label)

data = np.array(data)
labels = np.array(labels)

100%|██████████| 9144/9144 [00:11<00:00, 827.14it/s]


In [ ]:
data[0].shape

(224, 224, 3)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
lb = OneHotEncoder()
labels = lb.fit_transform(labels.reshape(-1, 1)).toarray()
print(labels.shape)

(8677, 101)


In [ ]:
#chia train, test, val
train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2


x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=1 - train_ratio)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))


#x_train = x_train.reshape(len(x_train), 256)
#x_val = x_val.reshape(len(x_val), 256)
#x_test = x_test.reshape(len(x_test), 256)
print('So luong tap train: ',len(x_train))
print('So luong tap val: ',len(x_val))
print('So luong tap test: ',len(x_test))

So luong tap train:  5206
So luong tap val:  1735
So luong tap test:  1736


In [ ]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
def mobilnet_block (x, filters, strides):

    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    return x

In [ ]:
# main part of the model
def get_model():
  input = Input(shape = (224,224,3))
  x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same', use_bias=False)(input)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)
  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first', padding='same')(x)
  x = Flatten()(x)
  output = Dense (units = 101, activation = 'softmax')(x)
  model = Model(inputs=input, outputs=output)
  #model.summary()
  return model

In [ ]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      864       
                                                                 
 batch_normalization (Batch  (None, 112, 112, 32)      128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwis  (None, 112, 112, 32)      288       
 eConv2D)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 32)      128   

# Yêu cầu 1: Với learning_rate = 0.0001, số lần lặp (epoch) =10
## Hãy xác định kết quả độ đo Accuracy trên các tập train-val-test cho các trường hợp sau
* cross entropy loss
* Focal Loss

In [ ]:
# Adam + learning_rate=1e-4, cross entropy, accuracy
model_1 = get_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model_1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_1.fit(x=x_train, y=y_train, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
163/163 [==============================] - 44s 102ms/step - loss: 4.2596 - accuracy: 0.1214 - val_loss: 4.4640 - val_accuracy: 0.1020
Epoch 2/10
163/163 [==============================] - 15s 90ms/step - loss: 3.9675 - accuracy: 0.1777 - val_loss: 4.3362 - val_accuracy: 0.1020
Epoch 3/10
163/163 [==============================] - 15s 90ms/step - loss: 3.8123 - accuracy: 0.2126 - val_loss: 4.3246 - val_accuracy: 0.0455
Epoch 4/10
163/163 [==============================] - 15s 90ms/step - loss: 3.6991 - accuracy: 0.2222 - val_loss: 4.4319 - val_accuracy: 0.0599
Epoch 5/10
163/163 [==============================] - 15s 90ms/step - loss: 3.5848 - accuracy: 0.2370 - val_loss: 3.9008 - val_accuracy: 0.2110
Epoch 6/10
163/163 [==============================] - 15s 91ms/step - loss: 3.4789 - accuracy: 0.2587 - val_loss: 3.4658 - val_accuracy: 0.2663
Epoch 7/10
163/163 [==============================] - 15s 91ms/step - loss: 3.3548 - accuracy: 0.2818 - val_loss: 3.3130 - val_accuracy

In [ ]:
model_1.evaluate(x_test, y_test)

55/55 [==============================] - 2s 27ms/step - loss: 3.1579 - accuracy: 0.3353


[3.1579484939575195, 0.33525344729423523]

In [ ]:
# Sgd + learning_rate=1e-4, cross entropy, accuracy
model_2 = get_model()
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
model_2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_2.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
163/163 [==============================] - 23s 95ms/step - loss: 4.8348 - accuracy: 0.0071 - val_loss: 4.6314 - val_accuracy: 0.0040
Epoch 2/10
163/163 [==============================] - 14s 89ms/step - loss: 4.7824 - accuracy: 0.0060 - val_loss: 4.6454 - val_accuracy: 0.0040
Epoch 3/10
163/163 [==============================] - 15s 90ms/step - loss: 4.7335 - accuracy: 0.0102 - val_loss: 4.6426 - val_accuracy: 0.0040
Epoch 4/10
163/163 [==============================] - 15s 90ms/step - loss: 4.6908 - accuracy: 0.0129 - val_loss: 4.6068 - val_accuracy: 0.0052
Epoch 5/10
163/163 [==============================] - 15s 90ms/step - loss: 4.6505 - accuracy: 0.0175 - val_loss: 4.5832 - val_accuracy: 0.0352
Epoch 6/10
163/163 [==============================] - 15s 89ms/step - loss: 4.6119 - accuracy: 0.0263 - val_loss: 4.5922 - val_accuracy: 0.0432
Epoch 7/10
163/163 [==============================] - 15s 90ms/step - loss: 4.5719 - accuracy: 0.0465 - val_loss: 5.1818 - val_accuracy:

In [ ]:
model_2.evaluate(x_test, y_test)

55/55 [==============================] - 1s 23ms/step - loss: 4.4685 - accuracy: 0.1048


[4.468520641326904, 0.10483870655298233]

In [ ]:
def focal_loss(y_true, y_pred, alpha=0.25, gamma=2.0):
    # Tính cross-entropy loss
    ce_loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)

    # Tính focal loss
    focal_loss = alpha * tf.math.pow(1 - y_pred, gamma) * ce_loss

    return tf.reduce_mean(focal_loss)

In [ ]:
# Adam + learning_rate=1e-4, focal loss, accuracy
model_3 = get_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model_3.compile(optimizer=optimizer, loss='categorical_focal_crossentropy', metrics=['accuracy'])

In [ ]:
model_3.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
163/163 [==============================] - 32s 98ms/step - loss: 1.0248 - accuracy: 0.1241 - val_loss: 1.0912 - val_accuracy: 0.1020
Epoch 2/10
163/163 [==============================] - 15s 91ms/step - loss: 0.9542 - accuracy: 0.1650 - val_loss: 1.0505 - val_accuracy: 0.1020
Epoch 3/10
163/163 [==============================] - 15s 91ms/step - loss: 0.9192 - accuracy: 0.1892 - val_loss: 1.0434 - val_accuracy: 0.0467
Epoch 4/10
163/163 [==============================] - 15s 90ms/step - loss: 0.8879 - accuracy: 0.2117 - val_loss: 1.0778 - val_accuracy: 0.0432
Epoch 5/10
163/163 [==============================] - 15s 92ms/step - loss: 0.8596 - accuracy: 0.2280 - val_loss: 0.9456 - val_accuracy: 0.1706
Epoch 6/10
163/163 [==============================] - 15s 91ms/step - loss: 0.8362 - accuracy: 0.2426 - val_loss: 0.8403 - val_accuracy: 0.2582
Epoch 7/10
163/163 [==============================] - 15s 90ms/step - loss: 0.8094 - accuracy: 0.2647 - val_loss: 0.8121 - val_accuracy:

In [ ]:
model_3.evaluate(x_test, y_test)

55/55 [==============================] - 1s 22ms/step - loss: 0.7599 - accuracy: 0.3128


[0.7599191069602966, 0.3127880096435547]

In [ ]:
# sgd + learning_rate=1e-4, focal loss, accuracy
model_4 = get_model()
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
model_4.compile(optimizer=optimizer, loss='categorical_focal_crossentropy', metrics=['accuracy'])

In [ ]:
model_4.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
163/163 [==============================] - 22s 97ms/step - loss: 1.1970 - accuracy: 0.0083 - val_loss: 1.1361 - val_accuracy: 0.0086
Epoch 2/10
163/163 [==============================] - 15s 89ms/step - loss: 1.1926 - accuracy: 0.0071 - val_loss: 1.1436 - val_accuracy: 0.0086
Epoch 3/10
163/163 [==============================] - 15s 90ms/step - loss: 1.1894 - accuracy: 0.0081 - val_loss: 1.1478 - val_accuracy: 0.0086
Epoch 4/10
163/163 [==============================] - 14s 89ms/step - loss: 1.1867 - accuracy: 0.0069 - val_loss: 1.1448 - val_accuracy: 0.0092
Epoch 5/10
163/163 [==============================] - 15s 89ms/step - loss: 1.1807 - accuracy: 0.0081 - val_loss: 1.1506 - val_accuracy: 0.0086
Epoch 6/10
163/163 [==============================] - 14s 88ms/step - loss: 1.1756 - accuracy: 0.0088 - val_loss: 1.1768 - val_accuracy: 0.0069
Epoch 7/10
163/163 [==============================] - 15s 89ms/step - loss: 1.1701 - accuracy: 0.0077 - val_loss: 1.1820 - val_accuracy:

In [ ]:
model_4.evaluate(x_test, y_test)

55/55 [==============================] - 1s 22ms/step - loss: 1.1753 - accuracy: 0.0092


[1.1752779483795166, 0.009216589853167534]

# Yêu cầu 2: Thay 3 layers tích chập 3x3, 3x3 và 1x1 bằng 1 layer có kích thước nhân tích chập là 7x7
* Hãy xác định kết quả trên tập test trong trường hợp: learning_rate = 0.0001, epoch=10, Optimizer = Adam, Weighted-class Cross Entropy Loss

In [ ]:
def get_af_model():
    input = Input(shape = (224, 224, 3))
    x = Conv2D(filters=32, kernel_size=7, strides=2, padding='same', use_bias=False)(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = mobilnet_block(x, filters = 128, strides = 2)
    x = mobilnet_block(x, filters = 128, strides = 1)
    x = mobilnet_block(x, filters = 256, strides = 2)
    x = mobilnet_block(x, filters = 256, strides = 1)
    x = mobilnet_block(x, filters = 512, strides = 2)
    for _ in range (5):
        x = mobilnet_block(x, filters = 512, strides = 2)
    x = mobilnet_block(x, filters = 1024, strides = 2)
    x = mobilnet_block(x, filters = 1024, strides = 1)
    x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first', padding='same')(x)
    x = Flatten()(x)
    output = Dense (units = 101, activation = 'softmax')(x)
    model = Model(inputs=input, outputs=output)

    return model

In [ ]:
cnt_class = np.sum(y_train, axis=0)
num_class = y_train.shape[1]

class_weights = [0 for _ in range(101)]
total_samples = len(y_train)
for i in range(num_class):
    class_weights[i] = total_samples / (num_class * cnt_class[i])

In [ ]:
model = get_af_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'], loss_weights=class_weights)

In [ ]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
163/163 [==============================] - 27s 76ms/step - loss: 0.8096 - accuracy: 0.1204 - val_loss: 0.8492 - val_accuracy: 0.0438
Epoch 2/10
163/163 [==============================] - 11s 70ms/step - loss: 0.7500 - accuracy: 0.1662 - val_loss: 0.8304 - val_accuracy: 0.0438
Epoch 3/10
163/163 [==============================] - 11s 70ms/step - loss: 0.7147 - accuracy: 0.1996 - val_loss: 0.8298 - val_accuracy: 0.0467
Epoch 4/10
163/163 [==============================] - 12s 72ms/step - loss: 0.6866 - accuracy: 0.2290 - val_loss: 0.8520 - val_accuracy: 0.0432
Epoch 5/10
163/163 [==============================] - 12s 71ms/step - loss: 0.6656 - accuracy: 0.2505 - val_loss: 0.7683 - val_accuracy: 0.1573
Epoch 6/10
163/163 [==============================] - 12s 72ms/step - loss: 0.6418 - accuracy: 0.2724 - val_loss: 0.6642 - val_accuracy: 0.2720
Epoch 7/10
163/163 [==============================] - 11s 70ms/step - loss: 0.6194 - accuracy: 0.2993 - val_loss: 0.6201 - val_accuracy:

In [ ]:
model.evaluate(x_test, y_test)

55/55 [==============================] - 1s 20ms/step - loss: 0.5847 - accuracy: 0.3439


[0.5847193598747253, 0.34389400482177734]